In [90]:
import pickle
import tensorflow as tf

import numpy as np

# ###### construct new dataset with captions and images
normData = pickle.load(open("normData","rb"))
captions_vectors = normData.captions
images_vectors = pickle.load(open('fc1_img','rb'))
labels = np.array(normData.labels)

captions_images_vectors = np.array([None]*len(captions_vectors))

for i in range(len(captions_images_vectors)):
    captions_images_vectors[i] = captions_vectors[i]*images_vectors[i]

##### assign
train_data = []
test_data = []
train_label = []
test_label = []
for i in range(1000):
    if i % 10 < 8:
        train_data.append(captions_images_vectors[i])
        train_label.append(labels[i])
    else:
        test_data.append(captions_images_vectors[i])
        test_label.append(labels[i])

train_data_nd = np.array(train_data)
train_label_nd = np.array(train_label)
test_data_nd = np.array(test_data)
test_label_nd = np.array(test_label)

######

In [91]:
captions_dim = 1024
x = tf.placeholder(tf.float32,[None,captions_dim])
W1 = tf.Variable(tf.zeros([captions_dim, 20]))
b1 = tf.Variable(tf.zeros([20]))
y1 = tf.matmul(x, W1) + b1
# output
y = y1
y_ = tf.placeholder(tf.float32,[None,20])

# cross_entropy
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y, labels=y_))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

for i in range(20000):
    sess.run(train_step, feed_dict={x: train_data, y_: train_label})
    if i % 200 == 0:
        print(i, ',accuracy:', sess.run(accuracy, feed_dict={x: test_data, y_: test_label}))

0 ,accuracy: 0.365
200 ,accuracy: 0.42
400 ,accuracy: 0.45
600 ,accuracy: 0.45
800 ,accuracy: 0.45
1000 ,accuracy: 0.455
1200 ,accuracy: 0.465
1400 ,accuracy: 0.46
1600 ,accuracy: 0.465
1800 ,accuracy: 0.46
2000 ,accuracy: 0.46
2200 ,accuracy: 0.46
2400 ,accuracy: 0.475
2600 ,accuracy: 0.475
2800 ,accuracy: 0.47
3000 ,accuracy: 0.47
3200 ,accuracy: 0.48
3400 ,accuracy: 0.48
3600 ,accuracy: 0.48
3800 ,accuracy: 0.48
4000 ,accuracy: 0.49
4200 ,accuracy: 0.49
4400 ,accuracy: 0.49
4600 ,accuracy: 0.495
4800 ,accuracy: 0.495
5000 ,accuracy: 0.495
5200 ,accuracy: 0.495
5400 ,accuracy: 0.495
5600 ,accuracy: 0.505
5800 ,accuracy: 0.495
6000 ,accuracy: 0.495
6200 ,accuracy: 0.495
6400 ,accuracy: 0.5
6600 ,accuracy: 0.5
6800 ,accuracy: 0.51
7000 ,accuracy: 0.515
7200 ,accuracy: 0.52
7400 ,accuracy: 0.525
7600 ,accuracy: 0.525
7800 ,accuracy: 0.525
8000 ,accuracy: 0.52
8200 ,accuracy: 0.515
8400 ,accuracy: 0.515
8600 ,accuracy: 0.515
8800 ,accuracy: 0.515
9000 ,accuracy: 0.515
9200 ,accuracy: 0.5

In [52]:
cap_vec = pickle.load(open('cap_vec','rb'))

In [53]:
def plot_with_labels(low_dim_embs, labels, filename='cap_vec.png'):
  assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
plot_only = 300
tsne = TSNE(perplexity=10, n_components=2, n_iter=5000,random_state=0)
low_dim_embs = tsne.fit_transform(list(captions_images_vectors[:]))
label = [labels[i].argmax() for i in range(plot_only)]
plot_with_labels(low_dim_embs, label)

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

In [83]:
cosine_similarity(fc2_pre[56],fc2_pre[80])

/Users/User/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
/Users/User/anaconda/lib/python3.5/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[ 0.9572987]], dtype=float32)

low_dim_embs = tsne.fit_transform(list(captions_images_vectors[:plot_only]))


In [97]:
pickle.dump(low_dim_embs,open('low_dim_embs','wb'))

In [95]:
pickle.dump(labels,open('labels','wb'))

In [102]:
low_dim_embs

array([[-25.61491316, -29.08214871],
       [ -7.42755174,  10.49207257],
       [-27.59600535, -33.78903937],
       ..., 
       [-23.07657498, -48.10741972],
       [-13.11786391, -40.86524063],
       [  1.47335793, -26.26037257]])

In [101]:
low_dim_embs.reshape((20,50,2))

array([[[-25.61491316, -29.08214871],
        [ -7.42755174,  10.49207257],
        [-27.59600535, -33.78903937],
        ..., 
        [-17.18890222, -45.2456405 ],
        [-12.33536968,  14.90747602],
        [-22.54915381, -15.83051094]],

       [[-32.12554332, -36.00458971],
        [  5.50203305,  -8.07926244],
        [-20.63265966, -46.91489231],
        ..., 
        [-20.46829662, -43.60111315],
        [-30.54012024, -38.95850374],
        [ -8.50607279,  -8.16972303]],

       [[ -3.94850966, -16.34663357],
        [-22.25452369, -29.23942882],
        [-25.49284092, -26.40627242],
        ..., 
        [-14.11207917, -41.0898278 ],
        [-11.35603308, -33.68211915],
        [-22.31035001, -42.46635771]],

       ..., 
       [[-16.02789651, -26.09524806],
        [ -0.95732389, -22.3326356 ],
        [-15.1020598 , -31.65143896],
        ..., 
        [  0.65651787, -12.53678577],
        [-17.1712848 , -35.60872887],
        [-24.94871051, -19.12408614]],

       [[  